In [4]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('project.db')



In [5]:
def sql(query):
    return pd.read_sql_query(query, conn)


In [6]:
sql("SELECT * FROM transactions LIMIT 5;")


,date_new,Id_check,ID_client,Count_products,Sum_payment
0,01/03/2016,2271145,104027,2.034,23.31
1,01/03/2016,2271145,104027,1.000,31.75
2,01/03/2016,2271145,104027,1.000,6.80
3,01/03/2016,2271145,104027,1.000,6.24
4,01/03/2016,2271145,104027,1.000,36.98


In [7]:
sql("""
WITH parsed_transactions AS (
    SELECT 
        strftime('%Y-%m', date(substr(date_new, 7, 4) || '-' || substr(date_new, 4, 2) || '-' || substr(date_new, 1, 2))) AS year_month,
        Sum_payment
    FROM transactions
    WHERE date(substr(date_new, 7, 4) || '-' || substr(date_new, 4, 2) || '-' || substr(date_new, 1, 2))
          BETWEEN '2015-06-01' AND '2016-06-01'
),
totals AS (
    SELECT 
        COUNT(*) AS total_operations,
        SUM(Sum_payment) AS total_amount
    FROM parsed_transactions
)

SELECT 
    p.year_month,
    COUNT(*) AS operations_in_month,
    ROUND(COUNT(*) * 100.0 / t.total_operations, 2) AS percent_of_operations,
    ROUND(SUM(p.Sum_payment), 2) AS amount_in_month,
    ROUND(SUM(p.Sum_payment) * 100.0 / t.total_amount, 2) AS percent_of_amount
FROM parsed_transactions p, totals t
GROUP BY p.year_month
ORDER BY p.year_month;
""")


,year_month,operations_in_month,percent_of_operations,amount_in_month,percent_of_amount
0,2015-06,3219,0.77,30101.16,0.76
1,2015-07,29697,7.09,274880.97,6.91
2,2015-08,28891,6.89,261817.75,6.58
3,2015-09,28088,6.70,260524.74,6.55
4,2015-10,29674,7.08,276558.69,6.95
5,2015-11,27337,6.52,252307.55,6.34
6,2015-12,30849,7.36,287656.22,7.23
7,2016-01,30383,7.25,275222.42,6.92
8,2016-02,48174,11.49,482447.38,12.13
9,2016-03,43268,10.32,428173.96,10.76


In [8]:
sql("""
WITH parsed_transactions AS (
    SELECT 
        t.ID_client,
        strftime('%Y-%m', date(substr(t.date_new, 7, 4) || '-' || substr(t.date_new, 4, 2) || '-' || substr(t.date_new, 1, 2))) AS year_month,
        t.Sum_payment,
        c.Gender
    FROM transactions t
    JOIN customer c ON t.ID_client = c.ID_client
    WHERE date(substr(t.date_new, 7, 4) || '-' || substr(t.date_new, 4, 2) || '-' || substr(t.date_new, 1, 2))
          BETWEEN '2015-06-01' AND '2016-06-01'
),
monthly_totals AS (
    SELECT 
        year_month,
        COUNT(*) AS total_operations,
        SUM(Sum_payment) AS total_sum
    FROM parsed_transactions
    GROUP BY year_month
)

SELECT 
    p.year_month,
    p.Gender,
    COUNT(*) AS operations_by_gender,
    ROUND(COUNT(*) * 100.0 / mt.total_operations, 2) AS percent_operations,
    ROUND(SUM(p.Sum_payment), 2) AS amount_by_gender,
    ROUND(SUM(p.Sum_payment) * 100.0 / mt.total_sum, 2) AS percent_amount
FROM parsed_transactions p
JOIN monthly_totals mt ON p.year_month = mt.year_month
GROUP BY p.year_month, p.Gender
ORDER BY p.year_month, p.Gender;
""")


,year_month,Gender,operations_by_gender,percent_operations,amount_by_gender,percent_amount
0,2015-06,None,73,2.27,652.69,2.17
1,2015-06,F,2233,69.37,21830.53,72.52
2,2015-06,M,913,28.36,7617.94,25.31
3,2015-07,None,646,2.18,5855.59,2.13
4,2015-07,F,21548,72.56,201413.62,73.27
5,2015-07,M,7503,25.27,67611.76,24.60
6,2015-08,None,688,2.38,6177.93,2.36
7,2015-08,F,21712,75.15,199609.74,76.24
8,2015-08,M,6491,22.47,56030.08,21.40
9,2015-09,None,793,2.82,7066.54,2.71


In [10]:
sql("PRAGMA table_info(customer);")


,cid,name,type,notnull,dflt_value,pk
0,0,Id_client,INTEGER,0,None,0
1,1,Total_amount,INTEGER,0,None,0
2,2,Gender,TEXT,0,None,0
3,3,Age,INTEGER,0,None,0
4,4,Count_city,INTEGER,0,None,0
5,5,Response_communcation,INTEGER,0,None,0
6,6,Communication_3month,INTEGER,0,None,0
7,7,Tenure,INTEGER,0,None,0


In [12]:
sql("""
-- Analyze client transactions by age groups (based on 'Age' column)
-- Part 1: Total number of transactions and total spend per group
-- Part 2: Quarterly average spend and share of total spend

WITH parsed_data AS (
    SELECT 
        t.ID_client,
        c.Age,
        strftime('%Y-%m', date(substr(t.date_new, 7, 4) || '-' || substr(t.date_new, 4, 2) || '-' || substr(t.date_new, 1, 2))) AS year_month,
        strftime('%m', date(substr(t.date_new, 7, 4) || '-' || substr(t.date_new, 4, 2) || '-' || substr(t.date_new, 1, 2))) AS month_val,
        t.Sum_payment,
        CASE 
            WHEN c.Age IS NULL THEN 'NA'
            WHEN c.Age < 10 THEN '0-9'
            WHEN c.Age < 20 THEN '10-19'
            WHEN c.Age < 30 THEN '20-29'
            WHEN c.Age < 40 THEN '30-39'
            WHEN c.Age < 50 THEN '40-49'
            WHEN c.Age < 60 THEN '50-59'
            WHEN c.Age < 70 THEN '60-69'
            ELSE '70+'
        END AS age_group
    FROM transactions t
    JOIN customer c ON t.ID_client = c.ID_client
    WHERE date(substr(t.date_new, 7, 4) || '-' || substr(t.date_new, 4, 2) || '-' || substr(t.date_new, 1, 2))
          BETWEEN '2015-06-01' AND '2016-06-01'
),
quarter_data AS (
    SELECT *,
        CASE 
            WHEN month_val IN ('01','02','03') THEN 'Q1'
            WHEN month_val IN ('04','05','06') THEN 'Q2'
            WHEN month_val IN ('07','08','09') THEN 'Q3'
            ELSE 'Q4'
        END AS quarter
    FROM parsed_data
)

-- part 1
SELECT 
    age_group,
    COUNT(*) AS total_operations,
    ROUND(SUM(Sum_payment), 2) AS total_amount
FROM quarter_data
GROUP BY age_group

UNION ALL

-- part 2
SELECT 
    age_group || ' - ' || quarter AS age_quarter,
    ROUND(AVG(Sum_payment), 2) AS avg_payment,
    ROUND(SUM(Sum_payment) * 100.0 / (SELECT SUM(Sum_payment) FROM quarter_data), 2) AS percent_of_total
FROM quarter_data
GROUP BY age_group, quarter
ORDER BY age_quarter;
""")


,age_group,total_operations,total_amount
0,0-9,1164.00,12047.81
1,0-9 - Q1,11.09,0.11
2,0-9 - Q2,10.94,0.05
3,0-9 - Q3,9.72,0.05
4,0-9 - Q4,9.54,0.09
5,10-19,14129.00,131021.16
6,10-19 - Q1,9.86,1.02
7,10-19 - Q2,9.46,0.97
8,10-19 - Q3,8.49,0.58
9,10-19 - Q4,8.95,0.72
